<a href="https://colab.research.google.com/github/daniel-usp/MBA-BIA-USP/blob/main/Estat%C3%ADstica%20para%20Ci%C3%AAncia%20de%20Dados/Estat%C3%ADstica_Ci%C3%AAncia_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de Caso -  Atributos que Influenciam o Valor dos Imóveis em São Paulo 🏙️

O mercado imobiliário de São Paulo é vasto e diversificado, abrangendo uma variedade de bairros, perfis de imóveis e públicos. Um investidor bem-informado busca compreender quais variáveis são determinantes para o valor total dos imóveis, de forma a realizar escolhas mais assertivas e maximizar o retorno sobre o investimento.

Imagine que você é um investidor interessado em alocar recursos em imóveis de São Paulo. Você deseja saber quais atributos, como área útil, número de quartos e proximidade de estações de metrô, influenciam mais diretamente o valor total dos imóveis. Para isso, você dispõe de um conjunto de dados coletados na plataforma Quinto Andar, com informações abrangentes sobre o mercado.

Nos exercícios a seguir, você será guiado através de uma série de análises para explorar os dados e testar hipóteses. Vamos iniciar essa jornada exploratória e estatística!

## Objetivo do Estudo 📊

Este estudo tem como objetivo principal realizar uma análise estatística descritiva e inferencial para investigar os fatores que afetam o valor dos imóveis em São Paulo. A partir desta análise, você poderá identificar variáveis que influenciam diretamente o valor dos imóveis e construir hipóteses robustas para análises futuras.

---

## Exercícios 📘

### 1. Estatística Descritiva: Primeiras Impressões 📈

#### Conceito de Estatística Descritiva

A estatística descritiva refere-se a métodos e procedimentos para sumarizar e organizar dados de forma a proporcionar uma visão geral e compreensível das características principais do conjunto de dados. Os parâmetros descritivos incluem:

- **Média**: Representa o valor médio de uma variável e é amplamente utilizada para identificar uma medida central, indicando a tendência geral dos valores.
- **Mediana**: Um parâmetro robusto contra valores extremos (ou outliers), que oferece uma alternativa à média para situações em que os dados apresentam grande variabilidade.
- **Desvio padrão**: Mede o grau de dispersão em torno da média, auxiliando a avaliação da homogeneidade dos dados.

💡 *Exercício*: Calcule essas estatísticas para as variáveis `valor_total`, `preco_m2`, `area_util`, `quartos`, `vagas` e `condominio`.




In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Carregamento dos dados
dados_imoveis = pd.read_excel('https://github.com/daniel-usp/MBA-BIA-USP/raw/main/Fundamentos%20de%20Python/dados_imoveis_SP_Aula.xlsx')

# Exibição das primeiras linhas para entender a estrutura dos dados
dados_imoveis.head()

,valor_total,preco_m2,area_util,quartos,vagas,condominio,suites,banheiros,piscina,academia,quadra,bairro,media_idh,media_gini,renda_percapita,estacao_prox,linha_prox,dist_metro
0,189900,5934.38,32,2,1,180,0,1,0,1,0,JOSE BONIFACIO,0.697182,0.410909,559.668182,SÃO MATEUS,PRATA,6822.823684
1,221110,5142.09,43,2,1,250,0,1,0,0,1,JOSE BONIFACIO,0.697182,0.410909,559.668182,CORINTHIANS-ITAQUERA,VERMELHA,4682.709841
2,219990,5116.05,43,1,1,300,0,1,1,0,0,JOSE BONIFACIO,0.697182,0.410909,559.668182,CORINTHIANS-ITAQUERA,VERMELHA,4579.311123
3,219990,5116.05,43,2,1,300,0,1,1,0,0,JOSE BONIFACIO,0.697182,0.410909,559.668182,CORINTHIANS-ITAQUERA,VERMELHA,4579.311123
4,220000,5365.85,41,2,1,300,0,1,1,1,1,JOSE BONIFACIO,0.697182,0.410909,559.668182,CORINTHIANS-ITAQUERA,VERMELHA,3313.918703


In [ ]:
# 1. Estatística Descritiva: Primeiras Impressões
# Cálculo de média, mediana e desvio padrão das variáveis
estatisticas_descritivas = dados_imoveis[['valor_total', 'preco_m2', 'area_util', 'quartos', 'vagas', 'condominio']].describe().T
estatisticas_descritivas['mediana'] = dados_imoveis[['valor_total', 'preco_m2', 'area_util', 'quartos', 'vagas', 'condominio']].median()
estatisticas_descritivas[['mean', 'mediana', 'std']]



,valor_total,preco_m2,area_util,quartos,vagas,condominio,suites,banheiros,piscina,academia,quadra,media_idh,media_gini,renda_percapita,dist_metro
count,2.259000e+03,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000,2259.000000
mean,5.545144e+05,7245.002032,71.065958,2.281098,1.236830,606.028774,0.652501,1.753431,0.376273,0.168659,0.316512,0.794526,0.455214,1610.743752,2761.009402
std,5.160549e+05,2887.327976,42.005088,0.632211,1.097605,554.350458,0.804167,1.024427,0.484557,0.374533,0.465218,0.058132,0.036816,1120.535302,2647.744236
min,8.000000e+04,1666.670000,22.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.692258,0.370968,522.515294,31.324895
25%,2.700000e+05,5151.645000,50.000000,2.000000,1.000000,350.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.748200,0.427333,870.821500,911.903103
50%,3.900000e+05,6578.950000,60.000000,2.000000,1.000000,485.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.788500,0.447250,1251.631739,1996.258152
75%,6.000000e+05,8636.360000,74.000000,3.000000,1.000000,650.000000,1.000000,2.000000,1.000000,0.000000,1.000000,0.826400,0.486000,1789.764000,3730.501883
max,3.950000e+06,18694.440000,420.000000,5.000000,39.000000,4800.000000,4.000000,8.000000,1.000000,1.000000,1.000000,0.944667,0.588889,6721.230000,26101.126557


### 2. Visualização das Distribuições 🖼️

A visualização de dados permite observar as distribuições e identificar padrões. Utilize gráficos adequados para as seguintes variáveis:

- **Histograma** para `valor_total`, `area_util` e `preco_m2` — ideal para examinar a frequência dos valores.
- **Gráfico de dispersão** entre `area_util` e `valor_total` — permite observar a relação entre área e valor.
- **Boxplot** de `valor_total` e `preco_m2` em função do número de `quartos` — possibilita comparar valores conforme o número de quartos.

🎨 *Pergunta*: Quais padrões emergem? O que eles sugerem em termos de comportamento das variáveis?



## Entendendo o BoxPlot

<img src='https://github.com/daniel-usp/MBA-BIA-USP/raw/main/Agente%20de%20IA/boxplot1.png' width=40%>

<img src='https://github.com/daniel-usp/MBA-BIA-USP/raw/main/Agente%20de%20IA/boxplot2.png' width=40%>



### 3. Comparação do Valor Médio por Bairro 🌆

Os bairros podem influenciar significativamente o preço dos imóveis devido a diferenças socioeconômicas e estruturais. Calcule a média e o desvio padrão do `valor_total` para cada bairro, e crie um gráfico de barras para visualizar as diferenças de preço.

*Exercício*: Selecione um bairro específico que você considera atrativo para investimentos e analise como o preço médio de seus imóveis se compara aos demais bairros.

*Pergunta*: Qual bairro apresenta, em média, imóveis com maior valor? Existe grande variabilidade de preços entre os bairros?



### Comparação de Médias e Testes de Hipótese para Diferentes Atributos dos Imóveis

A análise comparativa de médias é fundamental para compreender se variáveis categóricas, como proximidade ao metrô e localização em bairros diferentes, impactam significativamente o valor dos imóveis. Para isso, utilizamos testes de hipótese, que nos permitem avaliar se a diferença observada nas médias de dois grupos é estatisticamente significativa ou se pode ser atribuída ao acaso.

#### Teoria: Testes Paramétricos e Não-Paramétricos

Os testes de hipótese para comparação de médias são divididos em duas categorias principais:

1. **Testes Paramétricos**: Esses testes assumem que os dados seguem uma distribuição específica, geralmente a distribuição normal. Eles são mais poderosos em termos estatísticos quando essas suposições são atendidas. O **teste t de Student** para comparação de médias entre dois grupos é o exemplo clássico de teste paramétrico, sendo adequado para amostras que atendem às suposições de normalidade e homocedasticidade (ou seja, variâncias semelhantes entre os grupos).

2. **Testes Não-Paramétricos**: Quando as suposições de normalidade não são válidas, optamos por testes não-paramétricos, que são menos sensíveis a valores extremos e a distribuições atípicas. Um exemplo de teste não-paramétrico é o **teste de Mann-Whitney**, que compara as medianas de dois grupos e é indicado quando os dados não seguem a distribuição normal. Este teste utiliza a posição (ou ranking) dos valores para avaliar a diferença entre os grupos, sendo robusto para dados assimétricos ou com outliers.





### **Escolha da distribuição amostral adequada no teste paramétrico**
<img src='https://caelum-online-public.s3.amazonaws.com/1229-estatistica-parte3/01/img003.png' width=70%>

![Teste Bicaudal](https://caelum-online-public.s3.amazonaws.com/1229-estatistica-parte3/01/img005.png)

#### 4. Impacto da Proximidade ao Metrô no Valor Total dos Imóveis

Para avaliar se a proximidade a estações de metrô afeta o valor dos imóveis, faremos um teste de comparação de médias:

1. **Hipótese nula (\(Ho\))**: A média do valor dos imóveis próximos ao metrô (menos de 2 km) é igual à média dos imóveis mais distantes (mais de 2 km).
2. **Hipótese alternativa (\(H1\))**: A média do valor dos imóveis próximos ao metrô é diferente da média dos imóveis mais distantes.

Utilizaremos o **teste t de Student** para essa comparação, pois estamos assumindo que os dados seguem uma distribuição aproximadamente normal e que os grupos possuem variâncias semelhantes.

#### 5. Comparação de Preço por Metro Quadrado entre Dois Bairros

Para entender se o preço por metro quadrado difere significativamente entre dois bairros específicos, realizaremos um teste de hipótese não-paramétrico, dado que os preços por metro quadrado podem apresentar uma distribuição assimétrica.

1. **Hipótese nula (\(Ho\))**: A mediana dos preços por metro quadrado no Bairro 1 é igual à mediana dos preços no Bairro 2.
2. **Hipótese alternativa (\(H1\))**: A mediana dos preços por metro quadrado no Bairro 1 é diferente da mediana dos preços no Bairro 2.

Neste caso, utilizaremos o **teste de Mann-Whitney**, que permite comparar medianas de dois grupos independentemente de sua distribuição.

---

Essas análises de hipótese nos permitirão entender a influência de fatores geográficos e de localização no mercado imobiliário de São Paulo, possibilitando uma visão mais robusta para decisões de investimento.

### 6. Análise de Correlação 📊

Explore as relações entre as variáveis utilizando uma matriz de correlação. Isso permitirá identificar quais fatores possuem maior correlação com o `valor_total` dos imóveis. Crie um mapa de calor (heatmap) para visualizar a matriz.

- *Exercício*: Qual variável possui maior correlação com o `valor_total`? Tente identificar uma justificativa econômica para essa relação.

*Pergunta*: Quais variáveis apresentam maior correlação com o valor total do imóvel? Qual interpretação você faz disso?





### 7. Impacto do Índice de Desenvolvimento Humano (IDH) no Valor dos Imóveis 🧬

O IDH pode influenciar o valor dos imóveis, pois está relacionado à qualidade de vida dos moradores do bairro. Teste a hipótese de que o valor médio dos imóveis em bairros com IDH superior a 0.8 é maior.

1. Separe os dados em dois grupos: `alto_idh` (IDH > 0.8) e `baixo_idh` (IDH <= 0.8).
2. Realize um teste t para comparar os valores médios dos dois grupos.

💼 *Pergunta*: A hipótese foi confirmada? Qual é a relevância desse resultado para um investidor?

### 8. Análise de Influência de Amenidades no Valor dos Imóveis 🏊‍♀️🏋️‍♂️

Considere agora a influência de amenidades, como piscina, academia e quadra esportiva. Realize uma análise de variância (ANOVA) para verificar se a presença dessas amenidades impacta significativamente o `valor_total` dos imóveis.

1. Crie três grupos: imóveis com piscina, com academia e com quadra.
2. Calcule as médias e execute o teste ANOVA.

*Pergunta*: As amenidades influenciam o valor dos imóveis? Como esse conhecimento pode impactar suas decisões de investimento?



### 9. Estudo de Caso Final: Perfil do Imóvel Ideal para Investimento 💼

Baseando-se nas análises anteriores, proponha um perfil de imóvel ideal para investimento. Inclua os principais atributos que contribuem para o aumento do valor do imóvel, justificando suas escolhas com base nos resultados obtidos.

*Desafio*: Como você aplicaria esses resultados para realizar uma oferta em um imóvel específico de sua escolha?

---



## Conclusão 🎯

Ao completar este estudo, você terá desenvolvido uma compreensão sólida dos fatores que impactam o valor dos imóveis em São Paulo, com base em uma análise estatística detalhada. A partir dos resultados obtidos, você estará capacitado a tomar decisões de investimento mais embasadas e avaliar imóveis de maneira criteriosa.

Esse roteiro foi criado para fornecer uma compreensão acadêmica e aplicada sobre o valor dos imóveis. Boas análises e bons investimentos! 💰🏡